In [183]:
import os
import sys
from importnb import Notebook
import re
import pandas as pd
import numpy as np
import zipfile
from enum import Enum
import zlib
from dataclasses import dataclass, make_dataclass, field
from typing import Optional, Union, List

PKG_ROOT = os.path.dirname(os.path.realpath(os.getcwd()))
if not PKG_ROOT in sys.path:
    sys.path.append(PKG_ROOT)
    
with Notebook():
    from src.parserbase import SegBase, make_names, clean_duplicates

In [2]:
class Sav:
    
    def __init__(self, filename:str):
        self.filename = os.path.join(PKG_ROOT, 'saves', 'original', filename)
        self.zipfile = zipfile.ZipFile(self.filename)
        self.filelist = self.zipfile.namelist()
        self.files = [self.zipfile.read(_) for _ in self.filelist]
        self.gam_str = self.files[1].decode('latin')
        self.gam_version = re.findall(r'GAME\s*V\d+\.\d+', self.gam_str)[0]  #always 'GAMEV2.0'
        self.cre_version = re.findall(r'CRE\s*V\d+\.\d+', self.gam_str)[0]  #always 'CRE V1.0'
        
        

In [45]:
a = Sav('pstsave.pstsave')

In [233]:
f = os.path.join(PKG_ROOT, 'resources', 'PSTEE', 'Spell.csv')
df = pd.read_csv(f, index_col=0)


In [95]:
f = '/Users/CIDER/Library/Mobile Documents/com~apple~CloudDocs/游戏/BaldurGate1/PSTEE items.rtfd/txt.rtf'
with open(f, 'r') as source:
    t = source.read()

In [4]:
m = re.findall('[A-Z][A-Z ]{3,}:', t)
n = re.split('[A-Z][A-Z ]{3,}:', t)

In [5]:
p = [(_m, re.findall('(?P<ItemCode>[A-Z0-9]+)\s*-\s*(?P<Item>.*)', _n)) for _m, _n in zip(m, n[1:])]

In [6]:
q = pd.DataFrame([[_m[0][:-1].replace(' ', '_'), *_n] for _m in p for _n in _m[1]], columns=['Category', 'ItemCode', 'Item'])

In [7]:
q.Item = q.Item.apply(lambda x: re.sub('_+', '_', re.sub('[^a-zA-Z0-9]', '_', x.replace('+', 'PLUS')).strip('_')))

In [8]:
q[q[['Category', 'Item']].duplicated(keep=0)]

,Category,ItemCode,Item
31,ARROWS,AROW05,Arrow_of_Biting
38,ARROWS,AROW12,Arrow_of_Biting
43,AXES,AX1H03,Battle_Axe_PLUS2
45,AXES,AX1H05,Throwing_Axe_PLUS2
46,AXES,AX1H06,Throwing_Axe_PLUS2
...,...,...,...
1465,ONE_HANDED_SWORDS,SW1H41,Long_Sword_PLUS2
1481,ONE_HANDED_SWORDS,SW1H56,Scimitar
1516,ONE_HANDED_SWORDS,WA2S1H,Sword_of_Balduran
1534,TWO_HANDED_SWORDS,SW2H17,Gram_the_Sword_of_Grief_PLUS5_Throne_of_Bhaal


In [110]:
df = pd.DataFrame(re.findall(r'(?P<category>[A-Za-z][A-Za-z ]+)\t(?P<Item>.*)\t(?P<ItemCode>[A-Z0-9_]+)', t.replace('Hand to Hand', 'MELEE').replace('Short Sword', 'SHORTSWORD').replace('Unknown', 'Misc')))

In [127]:
df['Item'] = df[1].apply(lambda x: re.sub('_+', '_', re.sub('[^a-zA-Z0-9]', '_', x.replace('+', 'PLUS'))).strip('_').upper())

In [132]:
df

,ItemCode,Category,ItemSlot,Item
0,MINHP1,AMULET,AMULET,NO_NAME
1,AVEST,ARMOR,ARMOR,ANNAH_S_VEST
2,BOD_GP,ARMOR,ARMOR,BODICE_OF_THE_GODLESS_PRIEST
3,BOD_PQ,ARMOR,ARMOR,BODICE_OF_THE_PERILOUS_QUEST
4,ZERTH,ARMOR,ARMOR,DAK_KON_ZERTH_ARMOR
...,...,...,...,...
725,FDTEETH,MISC,MISC,TEETH_OF_THE_FIRE_DRAKE
726,VTEETH,MISC,MISC,TEETH_OF_THE_VIPER
727,VARGBITE,MISC,MISC,VARGOUILLE_BITE
728,BBWAND,WAND,WEAPON,BLACK_BARBED_BRANCH_WAND


In [145]:
g =df.groupby(['ItemSlot', 'Category'])

In [141]:
clean_duplicates(g.get_group(('ARMOR', 'ARMOR')))

,ItemCode,Category,ItemSlot,Item
0,AVEST,ARMOR,ARMOR,ANNAH_S_VEST
1,BOD_GP,ARMOR,ARMOR,BODICE_OF_THE_GODLESS_PRIEST
2,BOD_PQ,ARMOR,ARMOR,BODICE_OF_THE_PERILOUS_QUEST
3,ZERTH,ARMOR,ARMOR,DAK_KON_ZERTH_ARMOR
4,DUSTROBE,ARMOR,ARMOR,DUSTMAN_ROBES
5,CHASTITY,ARMOR,ARMOR,FALL_FROM_GRACE_S_CHASTITY_BODICE
6,JER_BR,ARMOR,ARMOR,JERKIN_OF_THE_BRAZEN_ROGUE
7,JER_FS,ARMOR,ARMOR,JERKIN_OF_THE_FLITTING_SHADOW
8,VERMIN,ARMOR,ARMOR,LOWER_PLANE_VERMIN_ARMOR
9,VHAILOR,ARMOR,ARMOR,VHAILOR_S_MERCYKILLER_ARMOR


In [162]:
df = df[['ItemSlot', 'Category', 'ItemCode', 'Item']]

In [163]:
df

,ItemSlot,Category,ItemCode,Item
0,AMULET,AMULET,MINHP1,NO_NAME
1,ARMOR,ARMOR,AVEST,ANNAH_S_VEST
2,ARMOR,ARMOR,BOD_GP,BODICE_OF_THE_GODLESS_PRIEST
3,ARMOR,ARMOR,BOD_PQ,BODICE_OF_THE_PERILOUS_QUEST
4,ARMOR,ARMOR,ZERTH,DAK_KON_ZERTH_ARMOR
...,...,...,...,...
725,MISC,MISC,FDTEETH,TEETH_OF_THE_FIRE_DRAKE
726,MISC,MISC,VTEETH,TEETH_OF_THE_VIPER
727,MISC,MISC,VARGBITE,VARGOUILLE_BITE
728,WEAPON,WAND,BBWAND,BLACK_BARBED_BRANCH_WAND


In [192]:
df1 = clean_duplicates(df, groupby=['ItemSlot', 'Category'])

In [195]:
e = make_names('Item', df1)

In [199]:
df1.to_csv(os.path.join(PKG_ROOT, 'resources', 'pstee', 'ITEM.csv'))

In [198]:
df1.Item.tolist()

['ACIDIC_SPONGE_BOLTS',
 'BOLTS_OF_ACHERON',
 'BOLTS_OF_KESSEK_THE_DEVOURER',
 'BOLTS_OF_WHISTLING_DOOM',
 'BOLTS_OF_WINCING',
 'JAGGED_BOLTS',
 'NORDOM_S_CROSSBOW_BOLTS',
 'RULE_OF_THREES_BOLTS',
 'WINGED_BOLTS',
 'ZEPHYR_BOLTS',
 'NO_NAME',
 'ANNAH_S_VEST',
 'BODICE_OF_THE_GODLESS_PRIEST',
 'BODICE_OF_THE_PERILOUS_QUEST',
 'DAK_KON_ZERTH_ARMOR',
 'DUSTMAN_ROBES',
 'FALL_FROM_GRACE_S_CHASTITY_BODICE',
 'JERKIN_OF_THE_BRAZEN_ROGUE',
 'JERKIN_OF_THE_FLITTING_SHADOW',
 'LOWER_PLANE_VERMIN_ARMOR',
 'VHAILOR_S_MERCYKILLER_ARMOR',
 'PET_LIM_LIM',
 'COPPER_COMMONS',
 'FESTHALL_ROOM_KEY',
 'ORNATE_BOX',
 'RAGS',
 'SMALL_RED_KEY',
 'TENEMENT_KEY',
 'TENEMENT_STOREROOM_KEY',
 'BEER_GOGGLES',
 'LENS_OF_CONFOUNDING',
 'LENS_OF_INHERENT_VICIOUSNESS',
 'LENS_OF_PERSPICACITY',
 'LENS_OF_SEEING_DOUBLE',
 'LENS_OF_THE_BAT',
 'LENS_OF_THE_HORIZON',
 'MAGNIFYING_LENS',
 'NORDOM_S_OCULAR',
 'OPTIX',
 'RETRACTABLE_SCOPE',
 'STEALTH_LENS',
 'TIME_S_EYEPIECE',
 'ABYSSAL_PIPE',
 'ACASTE_S_SKULL',
 'ADDER_S_T